Seleccionamos la ruta principal de trabajo

In [1]:
import os
from pathlib import Path

root_directory = Path("C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main")
os.chdir(root_directory)
print(root_directory)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main


## Imports


In [2]:
from typing import Any

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint


from anomalib import TaskType
from anomalib.data import MVTec
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import Padim, Patchcore

In [3]:
datamodule = MVTec(num_workers=0)
datamodule = MVTec(category="bottle")
datamodule.prepare_data()  
datamodule.setup()  

i, data = next(enumerate(datamodule.val_dataloader()))
print(data.keys())

dict_keys(['image_path', 'label', 'image', 'mask'])


In [4]:
callbacks = [
    ModelCheckpoint(
        mode="max",
        monitor="pixel_AUROC",
    ),
    EarlyStopping(
        monitor="pixel_AUROC",
        mode="max",
        patience=3,
    ),
]

In [5]:
engine = Engine(
    callbacks=callbacks,
    pixel_metrics="AUROC",
    accelerator="auto", 
    devices=1,
    logger=False,
)

In [6]:
model = Padim()
datamodule2 = MVTec(num_workers=5)

In [7]:
engine = Engine(task=TaskType.SEGMENTATION)
engine.fit(model=model, datamodule=datamodule2)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\core\optimizer.py:180: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | model                 | PadimModel               | 2.8 M 
1 | _transform            | Compose                  | 0     
2 | normalization_metrics | MinMax            

Training: |          | 0/? [00:00<?, ?it/s]

C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\loops\optimization\automatic.py:129: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


## Validation


In [8]:
test_results = engine.test(
    model=model,
    datamodule=datamodule2,
    ckpt_path=engine.trainer.checkpoint_callback.best_model_path,
)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
Restoring states from the checkpoint path at C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Padim\MVTec\bottle\v16\weights\lightning\model.ckpt
Loaded model weights from the checkpoint at C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Padim\MVTec\bottle\v16\weights\lightning\model.ckpt
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9215584397315979     │
│       image_F1Score       │    0.9945553541183472     │
│        pixel_AUROC        │    0.4970758855342865     │
│       pixel_F1Score       │    0.21142718195915222    │
└───────────────────────────┴───────────────────────────┘

## OpenVINO Inference

Now that we trained and tested a model, we could check a single inference result using OpenVINO inferencer object. This will demonstrate how a trained model could be used for inference.


Before we can use OpenVINO inference, let's export the model to OpenVINO format first.


In [9]:
engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
)

C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torch\onnx\_internal\jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torch\onnx\utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_graph_shape_type_inference(
C:\Users\fabiu\anaconda3\envs\Padim\lib\site-packages\torch\onnx\utils.py:1209: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp

WindowsPath('C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/results/Padim/MVTec/bottle/latest/weights/openvino/model.xml')

### Load the OpenVINO Model

By default, the output files are saved into `results` directory. Let's check where the OpenVINO model is stored.


In [11]:
output_path = Path(engine.trainer.default_root_dir)
print(output_path)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Padim\MVTec\bottle\latest


In [12]:
openvino_model_path = output_path / "weights" / "openvino" / "model.bin"
metadata = output_path / "weights" / "openvino" / "metadata.json"
print(openvino_model_path.exists(), metadata.exists())

True True


In [13]:
inferencer = OpenVINOInferencer(
    path=openvino_model_path, 
    metadata=metadata, 
    device="CPU", 
)

In [14]:
print(openvino_model_path)

C:\Users\fabiu\anaconda3\envs\Padim\anomalib-main\results\Padim\MVTec\bottle\latest\weights\openvino\model.bin


In [15]:
import os

# Define la ruta al  que contiene las imágenes
directory = "C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/datasets/MVTec/bottle/test/broken_large/"

# Vamos los archivos del directorio
files = os.listdir(directory)

# Nos quedamos solo con las imagenes 
image_files = [f for f in files if f.endswith('.png')]

# Ahora tenemos un bucle para hacer todas las iteraciones seguidas
for image_file in image_files:

    image_path = os.path.join(directory, image_file)
    predictions = inferencer.predict(image=image_path)
    print(f"Archivo: {image_file}, Score: {predictions.pred_score}, Label: {predictions.pred_label}")


Archivo: 000.png, Score: 0.6855475487171316, Label: 1
Archivo: 001.png, Score: 0.8263468634242828, Label: 1
Archivo: 002.png, Score: 0.8046006153687932, Label: 1
Archivo: 003.png, Score: 0.6663723507830751, Label: 1
Archivo: 004.png, Score: 0.6168878274149618, Label: 1
Archivo: 005.png, Score: 0.7161216140832029, Label: 1
Archivo: 006.png, Score: 0.6829037552112999, Label: 1
Archivo: 007.png, Score: 0.5332260895350084, Label: 1
Archivo: 008.png, Score: 0.8470386982132393, Label: 1
Archivo: 009.png, Score: 0.7765764355618838, Label: 1
Archivo: 010.png, Score: 0.7444249570793782, Label: 1
Archivo: 011.png, Score: 0.8184531087952696, Label: 1
Archivo: 012.png, Score: 0.7245415563110027, Label: 1
Archivo: 013.png, Score: 0.9661972741271863, Label: 1
Archivo: 014.png, Score: 0.645557343467518, Label: 1
Archivo: 015.png, Score: 0.8736774091931516, Label: 1
Archivo: 016.png, Score: 0.5785098392286285, Label: 1
Archivo: 017.png, Score: 0.754586612136651, Label: 1
Archivo: 018.png, Score: 0.614

In [16]:
import os

# Define la ruta al directorio que contiene las imágenes
directory = "C:/Users/fabiu/anaconda3/envs/Padim/anomalib-main/datasets/MVTec/bottle/test/good/"

# Vamos los archivos del directorio
files = os.listdir(directory)
# Nos quedamos solo con las imagenes 
image_files = [f for f in files if f.endswith('.png')]

# Ahora tenemos un bucle para hacer todas las iteraciones seguidas
for image_file in image_files:

    image_path = os.path.join(directory, image_file)
    predictions = inferencer.predict(image=image_path)
    print(f"Archivo: {image_file}, Score: {predictions.pred_score}, Label: {predictions.pred_label}")

Archivo: 000.png, Score: 0.4349900160096849, Label: 0
Archivo: 001.png, Score: 0.3853094983433047, Label: 0
Archivo: 002.png, Score: 0.38976184288884363, Label: 0
Archivo: 003.png, Score: 0.4054548954893262, Label: 0
Archivo: 004.png, Score: 0.3779426629275766, Label: 0
Archivo: 005.png, Score: 0.7357522943006067, Label: 1
Archivo: 006.png, Score: 0.6287362196979809, Label: 1
